In [3]:
import pandas as pd
import torch
import syft as sy
from torch import nn, optim
from sklearn.model_selection import train_test_split

# Hook PyTorch
hook = sy.TorchHook(torch)

# Create virtual workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

# Load the dataset
file_path = 'Datasets/creditcard.csv'
data = pd.read_csv(file_path)

# Convert DataFrame to PyTorch tensors
X = torch.tensor(data.drop('Class', axis=1).values, dtype=torch.float32)
y = torch.tensor(data['Class'].values, dtype=torch.float32)

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split training data for Bob and Alice
X_train_bob = X_train[:len(X_train)//2]
y_train_bob = y_train[:len(y_train)//2]
X_train_alice = X_train[len(X_train)//2:]
y_train_alice = y_train[len(y_train)//2:]

# Send the data to the workers
train_data_bob = sy.BaseDataset(X_train_bob.send(bob), y_train_bob.send(bob))
train_data_alice = sy.BaseDataset(X_train_alice.send(alice), y_train_alice.send(alice))

# Create federated datasets and federated data loader
federated_train_data = sy.FederatedDataset([train_data_bob, train_data_alice])
federated_train_loader = sy.FederatedDataLoader(federated_train_data, shuffle=True, batch_size=64)

# Define and initialize the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        model.send(data.location)  # Send the model to the data location

        optimizer.zero_grad()  # Reset gradients
        prediction = model(data)  # Predict
        loss = criterion(prediction.squeeze(), target)  # Calculate loss
        loss.backward()  # Backpropagate
        optimizer.step()  # Update weights

        model.get()  # Get the model back

        print('Epoch:', epoch, 'Loss:', loss.get())


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.